In [1]:
!head -3 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-19143

['2013-12-28 23:40:34', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1087', '3', '3', '8', '2148', '-1']
['2013-12-28 23:41:41', '8613770773315', '111.30.132.38', 'web', 'wspeed.qq.com', 'qq', '-', '1621', '5', '5', '8', '2148', '-1']
['2013-12-28 23:43:52', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1140', '5', '5', '8', '2148', '-1']


In [4]:
from collections import Counter


In [2]:
qq_5000_user_dpi_day1_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-*"
qq_5000_user_dpi_day2_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131229/part-*"
qq_5000_user_dpi_day3_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131230/part-*"

qq_5000_user_dpi_day4_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/20131231/part-*"

qq_5000_user_dpi_3days_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/*/part-*"

In [3]:
# train data by day
qq_5000_user_dpi_day1 = sc.textFile(qq_5000_user_dpi_day1_f).coalesce(50)  # repartition
qq_5000_user_dpi_day2 = sc.textFile(qq_5000_user_dpi_day2_f).coalesce(50)
qq_5000_user_dpi_day3 = sc.textFile(qq_5000_user_dpi_day3_f).coalesce(50)

# test data
qq_5000_user_dpi_day4 = sc.textFile(qq_5000_user_dpi_day4_f).coalesce(50)

# all 3 days' train data
qq_5000_user_dpi_3days = sc.textFile(qq_5000_user_dpi_3days_f).coalesce(50)  

## pairs

### positive pairs

In [261]:
train_positive_pairs_day1 = list(set(qq_5000_user_dpi_day1.map(lambda x: 'day1-' + eval(x)[1] + '-' + eval(x)[2]).collect()))
train_positive_pairs_day2 = list(set(qq_5000_user_dpi_day2.map(lambda x: 'day2-' + eval(x)[1] + '-' + eval(x)[2]).collect()))
train_positive_pairs_day3 = list(set(qq_5000_user_dpi_day3.map(lambda x: 'day3-' + eval(x)[1] + '-' + eval(x)[2]).collect()))

In [253]:
all_server_ip = sc.textFile\
("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_ip_1684/part-*").collect()

In [254]:
print(len(all_server_ip))

1684


In [256]:
test_positive_pairs_day4 = list(set(qq_5000_user_dpi_day4.filter(lambda x: eval(x)[2] in all_server_ip)\
                                    .map(lambda x: 'day4-' + eval(x)[1] + '-' + eval(x)[2]).collect()))

In [257]:
sc.parallelize(test_positive_pairs_day4).coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/test_positive_pairs/1231")

In [7]:
train_pos_pairs_3days_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_pos_pairs/part-*"

In [8]:
sc.parallelize(train_positive_pairs_day1).coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_positive_pairs/1228")
sc.parallelize(train_positive_pairs_day2).coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_positive_pairs/1229")
sc.parallelize(train_positive_pairs_day3).coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_positive_pairs/1230")

In [262]:
train_positive_pairs_day1 = sc.parallelize(train_positive_pairs_day1).coalesce(4)
train_positive_pairs_day2 = sc.parallelize(train_positive_pairs_day2).coalesce(4)
train_positive_pairs_day3 = sc.parallelize(train_positive_pairs_day3).coalesce(4)

In [263]:
test_positive_pairs_day4 = sc.parallelize(test_positive_pairs_day4).coalesce(4)

### unlabled pairs

In [334]:
train_unlabled_pairs_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_unlabled_pairs/part-*"
test_unlabled_pairs_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/test_unlabled_pairs/1231/part-*"

In [335]:
train_unlabled_pairs = sc.textFile(train_unlabled_pairs_f)
test_unlabled_pairs = sc.textFile(test_unlabled_pairs_f)

## features

### user-brand

In [184]:
user_brand_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_brand/part-*"
user_brand = sc.textFile(user_brand_f).map(lambda x: eval(x))

In [186]:
print(user_brand.take(3))

[('8615951423477', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('8613913928726', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('8613705177565', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [187]:
user_brand_dict = {}
for key,value in user_brand.collect():
    user_brand_dict[key] = value
print(user_brand_dict['8615951423477'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [198]:
tppd1_user_brand = train_positive_pairs_day1.map(lambda x: (x, user_brand_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd2_user_brand = train_positive_pairs_day2.map(lambda x: (x, user_brand_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd3_user_brand = train_positive_pairs_day3.map(lambda x: (x, user_brand_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [199]:
print(tppd3_user_brand.take(3))

[('day3-8615251712607-120.204.201.199', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day3-8613705147234-117.135.171.245', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day3-8613852217956-111.1.21.37', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [264]:
test_ppd4_user_brand = test_positive_pairs_day4.map(lambda x: (x, user_brand_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
print(test_ppd4_user_brand.take(3))

[('day4-8613655196538-117.135.171.239', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day4-8615150584084-120.204.200.174', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day4-8618351960371-120.198.196.16', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [336]:
train_up_user_brand = train_unlabled_pairs.map(lambda x: (x, user_brand_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
test_up_user_brand = test_unlabled_pairs.map(lambda x: (x, user_brand_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [390]:
print(train_up_user_brand.take(3))

[('day123-8615251759820-125.39.207.146', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), ('day123-8613851587583-183.195.233.46', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day123-8618761679626-111.11.28.170', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


### user-activetime

In [195]:
user_activetime_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_activetime/part-*"
user_activetime = sc.textFile(user_activetime_f).map(lambda x: eval(x))

In [196]:
print(user_activetime.take(3))

[('8613851429132', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 4, 0, 12, 0, 0, 0, 0, 39, 8, 0, 0, 0]), ('8615722826043', [0, 0, 0, 0, 0, 0, 0, 20, 0, 1, 1, 0, 1, 0, 0, 5, 3, 0, 0, 0, 32, 20, 2, 3]), ('8615105191420', [0, 0, 0, 0, 0, 0, 0, 24, 30, 25, 10, 26, 1, 0, 0, 0, 0, 6, 2, 1, 13, 62, 13, 36])]


In [197]:
user_activetime_dict = {}
for key,value in user_activetime.collect():
    user_activetime_dict[key] = value
print(user_activetime_dict['8615105191420'])

[0, 0, 0, 0, 0, 0, 0, 24, 30, 25, 10, 26, 1, 0, 0, 0, 0, 6, 2, 1, 13, 62, 13, 36]


In [200]:
tppd1_user_activetime = train_positive_pairs_day1.map(lambda x: (x, user_activetime_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd2_user_activetime = train_positive_pairs_day2.map(lambda x: (x, user_activetime_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd3_user_activetime = train_positive_pairs_day3.map(lambda x: (x, user_activetime_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [203]:
print(tppd3_user_activetime.take(3))

[('day3-8615251712607-120.204.201.199', [0, 0, 0, 0, 0, 0, 0, 0, 14, 2, 9, 11, 7, 1, 0, 0, 12, 0, 20, 12, 7, 0, 2, 0]), ('day3-8613705147234-117.135.171.245', [0, 0, 0, 0, 0, 0, 0, 484, 305, 182, 95, 316, 1, 11, 11, 9, 122, 385, 316, 395, 258, 450, 308, 25]), ('day3-8613852217956-111.1.21.37', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 54, 6])]


In [265]:
test_ppd4_user_activetime = test_positive_pairs_day4.map(lambda x: (x, user_activetime_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
print(test_ppd4_user_activetime.take(3))

[('day4-8613655196538-117.135.171.239', [0, 3, 0, 0, 0, 0, 0, 24, 12, 5, 0, 0, 92, 8, 1, 0, 3, 24, 49, 29, 10, 17, 72, 3]), ('day4-8615150584084-120.204.200.174', [83, 25, 3, 3, 21, 18, 7, 41, 28, 3, 12, 0, 0, 40, 38, 27, 78, 0, 7, 0, 30, 15, 66, 7]), ('day4-8618351960371-120.198.196.16', [1, 0, 3, 2, 0, 0, 4, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 30])]


In [338]:
train_up_user_activetime = train_unlabled_pairs.map(lambda x: (x, user_activetime_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
test_up_user_activetime = test_unlabled_pairs.map(lambda x: (x, user_activetime_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [339]:
print(train_up_user_activetime.take(3))

[('day123-8615251759820-125.39.207.146', [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 3, 0]), ('day123-8613851587583-183.195.233.46', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 3, 4, 8, 1, 0, 0, 0, 0, 0]), ('day123-8618761679626-111.11.28.170', [0, 0, 0, 0, 0, 0, 37, 15, 12, 86, 60, 0, 0, 9, 1, 0, 0, 0, 1, 1, 19, 94, 0, 0])]


### server-service

In [248]:
server_service_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_service_feature/part-*"
server_service = sc.textFile(server_service_f).map(lambda x: eval(x))

In [249]:
print(server_service.take(3))

[('14.17.19.77', [0, 0, 0, 0, 0, 0, 204, 0, 575, 0, 0, 0, 0, 0]), ('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 52, 0]), ('111.30.132.109', [0, 0, 0, 0, 0, 0, 0, 0, 325, 0, 0, 0, 2, 0])]


In [250]:
server_service_dict = {}
for key,value in server_service.collect():
    server_service_dict[key] = value
print(server_service_dict['14.17.19.77'])

[0, 0, 0, 0, 0, 0, 204, 0, 575, 0, 0, 0, 0, 0]


In [251]:
tppd1_server_service = train_positive_pairs_day1.map(lambda x: (x, server_service_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd2_server_service = train_positive_pairs_day2.map(lambda x: (x, server_service_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd3_server_service = train_positive_pairs_day3.map(lambda x: (x, server_service_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [252]:
print(tppd3_server_service.take(3))

[('day3-8615251712607-120.204.201.199', [0, 0, 0, 0, 0, 0, 0, 0, 35144, 0, 0, 0, 0, 0]), ('day3-8613705147234-117.135.171.245', [0, 0, 0, 0, 0, 0, 71, 0, 11457, 0, 0, 0, 0, 0]), ('day3-8613852217956-111.1.21.37', [0, 0, 0, 0, 0, 0, 18, 0, 455, 0, 0, 0, 17, 0])]


In [266]:
test_ppd4_server_service = test_positive_pairs_day4.map(lambda x: (x, server_service_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
print(test_ppd4_server_service.take(3))

[('day4-8613655196538-117.135.171.239', [0, 0, 0, 0, 3, 58, 0, 0, 44, 0, 9, 8422, 1, 0]), ('day4-8615150584084-120.204.200.174', [0, 0, 0, 0, 0, 0, 523, 0, 40352, 0, 0, 0, 0, 0]), ('day4-8618351960371-120.198.196.16', [0, 0, 0, 0, 0, 0, 7615, 0, 121, 0, 0, 0, 0, 0])]


In [344]:
train_up_server_service  = train_unlabled_pairs.map(lambda x: (x, server_service_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
test_up_server_service = test_unlabled_pairs.map(lambda x: (x, server_service_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [345]:
print(test_up_server_service.take(3))

[('day4-8613851678445-27.115.124.156', [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), ('day4-8613512544256-221.194.36.147', [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0]), ('day4-8615951816800-183.136.158.176', [0, 0, 0, 0, 0, 0, 84, 0, 101, 0, 0, 0, 0, 0])]


### server-sub_service

In [216]:
server_subservice_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_sub_service_feature/part-*"
server_subservice = sc.textFile(server_subservice_f).map(lambda x: eval(x))

In [217]:
print(server_subservice.take(3))

[('14.17.19.77', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 204, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 575, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 52, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('111.30.132.109', [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 325, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [218]:
server_subservice_dict = {}
for key,value in server_subservice.collect():
    server_subservice_dict[key] = value
print(server_subservice_dict['14.17.19.77'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 204, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 575, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [226]:
tppd1_server_subservice = train_positive_pairs_day1.map(lambda x: (x, server_subservice_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd2_server_subservice = train_positive_pairs_day2.map(lambda x: (x, server_subservice_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd3_server_subservice = train_positive_pairs_day3.map(lambda x: (x, server_subservice_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [227]:
print(tppd2_server_subservice.take(3))

[('day2-8615850662096-120.196.210.154', [0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1151, 4, 0, 0, 0, 0, 0, 0, 0, 0]), ('day2-8613770963797-120.198.203.152', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12083, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 53, 0, 0, 0, 0, 0, 3, 0, 0, 0]), ('day2-8615951875805-120.198.196.16', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 7614, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 121, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [267]:
test_ppd4_server_subservice = test_positive_pairs_day4.map(lambda x: (x, server_subservice_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
print(test_ppd4_server_subservice.take(3))

[('day4-8613655196538-117.135.171.239', [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8422, 0, 9, 58, 0, 0, 0, 0, 0, 0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 3]), ('day4-8615150584084-120.204.200.174', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 523, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40352, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day4-8618351960371-120.198.196.16', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 7614, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 121, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [346]:
train_up_server_subservice  = train_unlabled_pairs.map(lambda x: (x, server_subservice_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
test_up_server_subservice = test_unlabled_pairs.map(lambda x: (x, server_subservice_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [347]:
print(test_up_server_subservice.take(3))

[('day4-8613851678445-27.115.124.156', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day4-8613512544256-221.194.36.147', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day4-8615951816800-183.136.158.176', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 0, 0, 0, 0, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 101, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


### server-activetime

In [204]:
server_activetime_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_active_time/part-*"
server_activetime = sc.textFile(server_activetime_f).map(lambda x: eval(x))

In [205]:
print(server_activetime.take(3))

[('14.17.19.77', [41, 12, 7, 5, 10, 11, 8, 20, 29, 25, 44, 47, 29, 38, 35, 34, 38, 30, 36, 52, 43, 59, 51, 75]), ('111.10.27.180', [1, 0, 2, 3, 3, 0, 1, 3, 4, 1, 6, 2, 1, 6, 2, 2, 2, 1, 7, 1, 0, 1, 4, 0]), ('111.30.132.109', [0, 0, 6, 12, 0, 4, 18, 35, 51, 130, 67, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0])]


In [206]:
server_activetime_dict = {}
for key,value in server_activetime.collect():
    server_activetime_dict[key] = value
print(server_activetime_dict['14.17.19.77'])

[41, 12, 7, 5, 10, 11, 8, 20, 29, 25, 44, 47, 29, 38, 35, 34, 38, 30, 36, 52, 43, 59, 51, 75]


In [209]:
tppd1_server_activetime = train_positive_pairs_day1.map(lambda x: (x, server_activetime_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd2_server_activetime = train_positive_pairs_day2.map(lambda x: (x, server_activetime_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
tppd3_server_activetime = train_positive_pairs_day3.map(lambda x: (x, server_activetime_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [210]:
print(tppd2_server_activetime.take(3))

[('day2-8615850662096-120.196.210.154', [81, 36, 8, 6, 6, 24, 5, 45, 77, 58, 61, 42, 73, 55, 76, 85, 63, 31, 24, 77, 55, 28, 86, 70]), ('day2-8613770963797-120.198.203.152', [305, 185, 107, 90, 78, 94, 191, 454, 639, 684, 665, 702, 646, 664, 611, 618, 716, 786, 722, 756, 676, 623, 630, 497]), ('day2-8615951875805-120.198.196.16', [295, 202, 158, 162, 231, 132, 205, 325, 322, 390, 354, 391, 421, 373, 353, 391, 392, 348, 359, 366, 445, 381, 397, 343])]


In [268]:
test_ppd4_server_activetime = test_positive_pairs_day4.map(lambda x: (x, server_activetime_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
print(test_ppd4_server_activetime.take(3))

[('day4-8613655196538-117.135.171.239', [222, 297, 177, 111, 99, 140, 122, 327, 412, 527, 530, 400, 462, 372, 429, 415, 585, 424, 483, 417, 486, 410, 268, 422]), ('day4-8615150584084-120.204.200.174', [2262, 1135, 999, 746, 755, 720, 1023, 1337, 1740, 1946, 2051, 1922, 2027, 1975, 1956, 2028, 1996, 1835, 1998, 1830, 1974, 2204, 2283, 2133]), ('day4-8618351960371-120.198.196.16', [295, 202, 158, 162, 231, 132, 205, 325, 322, 390, 354, 391, 421, 373, 353, 391, 392, 348, 359, 366, 445, 381, 397, 343])]


In [348]:
train_up_server_activetime  = train_unlabled_pairs.map(lambda x: (x, server_activetime_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))
test_up_server_activetime = test_unlabled_pairs.map(lambda x: (x, server_activetime_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

In [349]:
print(train_up_server_activetime.take(3))

[('day123-8615251759820-125.39.207.146', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('day123-8613851587583-183.195.233.46', [57, 0, 0, 0, 0, 0, 290, 72, 22, 41, 91, 142, 37, 13, 49, 2, 37, 9, 103, 57, 280, 362, 26, 9]), ('day123-8618761679626-111.11.28.170', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 6, 0, 0, 1, 0])]


### user_diversity #?

In [277]:
# user: qq_server(category)
user_diversity_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))
user_diversity_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))
user_diversity_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))
user_diversity_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))

In [271]:
'''user_diversity_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_diversity/1228")
user_diversity_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_diversity/1229")
user_diversity_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_diversity/1230")'''
user_diversity_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_diversity/1231")

In [278]:
user_diversity_day4.take(3)

[('8613851429132', 5), ('8613913395739', 7), ('8618260098095', 1)]

In [126]:
user_diversity_day1_dict = {}
for key, value in user_diversity_day1.collect():
    user_diversity_day1_dict[key] = value
user_diversity_day2_dict = {}
for key, value in user_diversity_day2.collect():
    user_diversity_day2_dict[key] = value
user_diversity_day3_dict = {}
for key, value in user_diversity_day3.collect():
    user_diversity_day3_dict[key] = value

In [280]:
user_diversity_day4_dict = {}
for key, value in user_diversity_day4.collect():
    user_diversity_day4_dict[key] = value
print(user_diversity_day4_dict['8613913395739'])

7


In [148]:
tppd1_user_diversity = train_positive_pairs_day1.map(lambda x: (x, user_diversity_day1_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd2_user_diversity = train_positive_pairs_day2.map(lambda x: (x, user_diversity_day2_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd3_user_diversity = train_positive_pairs_day3.map(lambda x: (x, user_diversity_day3_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [150]:
tppd1_user_diversity.take(3)

[('day1-8615150560352-111.30.128.27', [29]),
 ('day1-8613813010581-117.135.169.19', [12]),
 ('day1-8615951600106-117.135.169.19', [3])]

In [283]:
test_ppd4_user_diversity = test_positive_pairs_day4.map(lambda x: (x, user_diversity_day4_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
print(test_ppd4_user_diversity.take(3))

[('day4-8613655196538-117.135.171.239', [32]), ('day4-8615150584084-120.204.200.174', [39]), ('day4-8618351960371-120.198.196.16', [10])]


### user_degree #?

In [281]:
# user:degree(qq_server_times)
user_degree_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1],1)).groupByKey().mapValues(len)
user_degree_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1],1)).groupByKey().mapValues(len)
user_degree_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1],1)).groupByKey().mapValues(len)

user_degree_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1],1)).groupByKey().mapValues(len)

In [27]:
user_degree_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_degree/1228")
user_degree_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_degree/1229")
user_degree_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_degree/1230")

In [282]:
user_degree_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_degree/1231")

In [52]:
user_degree_day3.take(3)

[('8615722826043', 24), ('8613851429132', 47), ('8615105191420', 71)]

In [127]:
user_degree_day1_dict = {}
for key, value in user_degree_day1.collect():
    user_degree_day1_dict[key] = value
user_degree_day2_dict = {}
for key, value in user_degree_day2.collect():
    user_degree_day2_dict[key] = value
user_degree_day3_dict = {}
for key, value in user_degree_day3.collect():
    user_degree_day3_dict[key] = value

In [89]:
user_degree_day3_dict['8615105191420']

71

In [284]:
user_degree_day4_dict = {}
for key, value in user_degree_day4.collect():
    user_degree_day4_dict[key] = value

In [151]:
tppd1_user_degree = train_positive_pairs_day1.map(lambda x: (x, user_degree_day1_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd2_user_degree = train_positive_pairs_day2.map(lambda x: (x, user_degree_day2_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd3_user_degree = train_positive_pairs_day3.map(lambda x: (x, user_degree_day3_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [152]:
tppd1_user_degree.take(3)

[('day1-8615150560352-111.30.128.27', [74]),
 ('day1-8613813010581-117.135.169.19', [72]),
 ('day1-8615951600106-117.135.169.19', [124])]

In [285]:
test_ppd4_user_degree = test_positive_pairs_day4.map(lambda x: (x, user_degree_day4_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [286]:
print(test_ppd4_user_degree.take(3))

[('day4-8613655196538-117.135.171.239', [225]), ('day4-8615150584084-120.204.200.174', [200]), ('day4-8618351960371-120.198.196.16', [33])]


### user_duration

In [287]:
user_duration_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
user_duration_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
user_duration_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

user_duration_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

In [44]:
user_duration_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_duration/1228")
user_duration_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_duration/1229")
user_duration_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_duration/1230")

In [288]:
user_duration_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_duration/1231")

In [289]:
user_duration_day4.take(3)

[('8613851429132', 60463),
 ('8613913395739', 181016),
 ('8618260098095', 151488)]

In [128]:
user_duration_day1_dict = {}
for key, value in user_duration_day1.collect():
    user_duration_day1_dict[key] = value
user_duration_day2_dict = {}
for key, value in user_duration_day2.collect():
    user_duration_day2_dict[key] = value
user_duration_day3_dict = {}
for key, value in user_duration_day3.collect():
    user_duration_day3_dict[key] = value

In [290]:
user_duration_day4_dict = {}
for key, value in user_duration_day4.collect():
    user_duration_day4_dict[key] = value

In [153]:
tppd1_user_duration = train_positive_pairs_day1.map(lambda x: (x, user_duration_day1_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd2_user_duration = train_positive_pairs_day2.map(lambda x: (x, user_duration_day2_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd3_user_duration = train_positive_pairs_day3.map(lambda x: (x, user_duration_day3_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [154]:
tppd1_user_duration.take(3)

[('day1-8615150560352-111.30.128.27', [560262]),
 ('day1-8613813010581-117.135.169.19', [168686]),
 ('day1-8615951600106-117.135.169.19', [133647])]

In [297]:
test_ppd4_user_duration = test_positive_pairs_day4.map(lambda x: (x, user_duration_day4_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [298]:
test_ppd4_user_duration.take(3)

[('day4-8613655196538-117.135.171.239', [1508421]),
 ('day4-8615150584084-120.204.200.174', [1801203]),
 ('day4-8618351960371-120.198.196.16', [106544])]

### user_uppacket

In [293]:
user_uppacket_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
user_uppacket_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
user_uppacket_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

user_uppacket_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

In [48]:
user_uppacket_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_uppacket/1228")
user_uppacket_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_uppacket/1229")
user_uppacket_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_uppacket/1230")

In [294]:
user_uppacket_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_uppacket/1231")

In [47]:
user_uppacket_day3.take(3)

[('8615722826043', 194), ('8613851429132', 207), ('8615105191420', 465)]

In [129]:
user_uppacket_day1_dict = {}
for key, value in user_uppacket_day1.collect():
    user_uppacket_day1_dict[key] = value
user_uppacket_day2_dict = {}
for key, value in user_uppacket_day2.collect():
    user_uppacket_day2_dict[key] = value
user_uppacket_day3_dict = {}
for key, value in user_uppacket_day3.collect():
    user_uppacket_day3_dict[key] = value

In [295]:
user_uppacket_day4_dict = {}
for key, value in user_uppacket_day4.collect():
    user_uppacket_day4_dict[key] = value

In [155]:
tppd1_user_uppacket = train_positive_pairs_day1.map(lambda x: (x, user_uppacket_day1_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd2_user_uppacket = train_positive_pairs_day2.map(lambda x: (x, user_uppacket_day2_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd3_user_uppacket = train_positive_pairs_day3.map(lambda x: (x, user_uppacket_day3_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [156]:
tppd1_user_uppacket.take(3)

[('day1-8615150560352-111.30.128.27', [538]),
 ('day1-8613813010581-117.135.169.19', [353]),
 ('day1-8615951600106-117.135.169.19', [1024])]

In [301]:
test_ppd4_user_uppacket = test_positive_pairs_day4.map(lambda x: (x, user_uppacket_day4_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [302]:
test_ppd4_user_uppacket.take(3)

[('day4-8613655196538-117.135.171.239', [1648]),
 ('day4-8615150584084-120.204.200.174', [2282]),
 ('day4-8618351960371-120.198.196.16', [149])]

### user-downpacket

In [303]:
user_downpacket_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
user_downpacket_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
user_downpacket_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

user_downpacket_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

In [51]:
user_downpacket_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_downpacket/1228")
user_downpacket_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_downpacket/1229")
user_downpacket_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_downpacket/1230")

In [304]:
user_downpacket_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_downpacket/1231")

In [50]:
user_downpacket_day3.take(3)

[('8615722826043', 118), ('8613851429132', 197), ('8615105191420', 448)]

In [130]:
user_downpacket_day1_dict = {}
for key, value in user_downpacket_day1.collect():
    user_downpacket_day1_dict[key] = value
user_downpacket_day2_dict = {}
for key, value in user_downpacket_day2.collect():
    user_downpacket_day2_dict[key] = value
user_downpacket_day3_dict = {}
for key, value in user_downpacket_day3.collect():
    user_downpacket_day3_dict[key] = value

In [305]:
user_downpacket_day4_dict = {}
for key, value in user_downpacket_day4.collect():
    user_downpacket_day4_dict[key] = value

In [157]:
tppd1_user_downpacket = train_positive_pairs_day1.map(lambda x: (x, user_downpacket_day1_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd2_user_downpacket = train_positive_pairs_day2.map(lambda x: (x, user_downpacket_day2_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)
tppd3_user_downpacket = train_positive_pairs_day3.map(lambda x: (x, user_downpacket_day3_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [158]:
tppd1_user_downpacket.take(3)

[('day1-8615150560352-111.30.128.27', [421]),
 ('day1-8613813010581-117.135.169.19', [299]),
 ('day1-8615951600106-117.135.169.19', [469])]

In [306]:
test_ppd4_user_downpacket = test_positive_pairs_day4.map(lambda x: (x, user_downpacket_day4_dict[x.split('-')[1]]))\
.groupByKey().mapValues(list)

In [307]:
test_ppd4_user_downpacket.take(3)

[('day4-8613655196538-117.135.171.239', [1374]),
 ('day4-8615150584084-120.204.200.174', [1643]),
 ('day4-8618351960371-120.198.196.16', [109])]

### server-diversity

In [308]:
# server: users(kinds of users)
server_diversity_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))
server_diversity_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))
server_diversity_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))

server_diversity_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0],len(x[1])))

In [58]:
server_diversity_day3.take(3)

[('14.17.19.77', 196), ('111.10.27.180', 8), ('111.30.132.109', 115)]

In [60]:
server_diversity_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_diversity/1228")
server_diversity_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_diversity/1229")
server_diversity_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_diversity/1230")

In [309]:
server_diversity_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_diversity/1231")

In [131]:
server_diversity_day1_dict = {}
for key, value in server_diversity_day1.collect():
    server_diversity_day1_dict[key] = value
server_diversity_day2_dict = {}
for key, value in server_diversity_day2.collect():
    server_diversity_day2_dict[key] = value
server_diversity_day3_dict = {}
for key, value in server_diversity_day3.collect():
    server_diversity_day3_dict[key] = value

In [310]:
server_diversity_day4_dict = {}
for key, value in server_diversity_day4.collect():
    server_diversity_day4_dict[key] = value

In [159]:
tppd1_server_diversity = train_positive_pairs_day1.map(lambda x: (x, server_diversity_day1_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd2_server_diversity = train_positive_pairs_day2.map(lambda x: (x, server_diversity_day2_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd3_server_diversity = train_positive_pairs_day3.map(lambda x: (x, server_diversity_day3_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

In [311]:
test_ppd4_server_diversity = test_positive_pairs_day4.map(lambda x: (x, server_diversity_day4_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

In [312]:
test_ppd4_server_diversity.take(3)

[('day4-8613655196538-117.135.171.239', [325]),
 ('day4-8615150584084-120.204.200.174', [1571]),
 ('day4-8618351960371-120.198.196.16', [1009])]

In [160]:
tppd1_server_diversity.take(3)

[('day1-8615150560352-111.30.128.27', [744]),
 ('day1-8613813010581-117.135.169.19', [2612]),
 ('day1-8615951600106-117.135.169.19', [2612])]

### server-degree

In [313]:
# server: users(times)
server_degree_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], 1)).groupByKey().mapValues(len)
server_degree_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], 1)).groupByKey().mapValues(len)
server_degree_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], 1)).groupByKey().mapValues(len)

server_degree_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], 1)).groupByKey().mapValues(len)

In [65]:
server_degree_day3.take(3)

[('14.17.19.77', 265), ('111.10.27.180', 12), ('111.30.132.109', 322)]

In [66]:
server_degree_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_degree/1228")
server_degree_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_degree/1229")
server_degree_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_degree/1230")

In [314]:
server_degree_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_degree/1231")

In [132]:
server_degree_day1_dict = {}
for key, value in server_degree_day1.collect():
    server_degree_day1_dict[key] = value
server_degree_day2_dict = {}
for key, value in server_degree_day2.collect():
    server_degree_day2_dict[key] = value
server_degree_day3_dict = {}
for key, value in server_degree_day3.collect():
    server_degree_day3_dict[key] = value

In [315]:
server_degree_day4_dict = {}
for key, value in server_degree_day4.collect():
    server_degree_day4_dict[key] = value

In [161]:
tppd1_server_degree = train_positive_pairs_day1.map(lambda x: (x, server_degree_day1_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd2_server_degree = train_positive_pairs_day2.map(lambda x: (x, server_degree_day2_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd3_server_degree = train_positive_pairs_day3.map(lambda x: (x, server_degree_day3_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

In [162]:
tppd1_server_degree.take(3)

[('day1-8615150560352-111.30.128.27', [1192]),
 ('day1-8613813010581-117.135.169.19', [29573]),
 ('day1-8615951600106-117.135.169.19', [29573])]

In [316]:
test_ppd4_server_degree = test_positive_pairs_day4.map(lambda x: (x, server_degree_day4_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

### server-duration

In [318]:
server_duration_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
server_duration_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
server_duration_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

server_duration_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

In [71]:
server_duration_day2.take(3)

[('14.17.19.77', 1917887), ('111.10.27.180', 232371), ('111.30.132.109', 4072)]

In [72]:
server_duration_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_duration/1228")
server_duration_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_duration/1229")
server_duration_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_duration/1230")

In [319]:
server_duration_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_duration/1231")

In [163]:
server_duration_day1_dict = {}
for key, value in server_duration_day1.collect():
    server_duration_day1_dict[key] = value
server_duration_day2_dict = {}
for key, value in server_duration_day2.collect():
    server_duration_day2_dict[key] = value
server_duration_day3_dict = {}
for key, value in server_duration_day3.collect():
    server_duration_day3_dict[key] = value

In [320]:
server_duration_day4_dict = {}
for key, value in server_duration_day4.collect():
    server_duration_day4_dict[key] = value

In [164]:
tppd1_server_duration = train_positive_pairs_day1.map(lambda x: (x, server_duration_day1_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd2_server_duration = train_positive_pairs_day2.map(lambda x: (x, server_duration_day2_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd3_server_duration = train_positive_pairs_day3.map(lambda x: (x, server_duration_day3_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

In [165]:
tppd1_server_duration.take(3)

[('day1-8615150560352-111.30.128.27', [9376983]),
 ('day1-8613813010581-117.135.169.19', [76969374]),
 ('day1-8615951600106-117.135.169.19', [76969374])]

In [321]:
test_ppd4_server_duration = test_positive_pairs_day4.map(lambda x: (x, server_duration_day4_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

### server-uppacket

In [322]:
server_uppacket_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
server_uppacket_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
server_uppacket_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

server_uppacket_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

In [74]:
server_uppacket_day2.take(3)

[('14.17.19.77', 1626), ('111.10.27.180', 1403), ('111.30.132.109', 14)]

In [75]:
server_uppacket_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_uppacket/1228")
server_uppacket_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_uppacket/1229")
server_uppacket_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_uppacket/1230")

In [323]:
server_uppacket_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_uppacket/1231")

In [134]:
server_uppacket_day1_dict = {}
for key, value in server_uppacket_day1.collect():
    server_uppacket_day1_dict[key] = value
server_uppacket_day2_dict = {}
for key, value in server_uppacket_day2.collect():
    server_uppacket_day2_dict[key] = value
server_uppacket_day3_dict = {}
for key, value in server_uppacket_day3.collect():
    server_uppacket_day3_dict[key] = value

In [324]:
server_uppacket_day4_dict = {}
for key, value in server_uppacket_day4.collect():
    server_uppacket_day4_dict[key] = value

In [166]:
tppd1_server_uppacket = train_positive_pairs_day1.map(lambda x: (x, server_uppacket_day1_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd2_server_uppacket = train_positive_pairs_day2.map(lambda x: (x, server_uppacket_day2_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd3_server_uppacket = train_positive_pairs_day3.map(lambda x: (x, server_uppacket_day3_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

In [167]:
tppd1_server_uppacket.take(3)

[('day1-8615150560352-111.30.128.27', [5236]),
 ('day1-8613813010581-117.135.169.19', [147879]),
 ('day1-8615951600106-117.135.169.19', [147879])]

In [325]:
test_ppd4_server_uppacket = test_positive_pairs_day4.map(lambda x: (x, server_uppacket_day4_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

### server-downpacket

In [326]:
server_downpacket_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
server_downpacket_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))
server_downpacket_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))

server_downpacket_day4 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], sum(x[1])))


In [77]:
server_downpacket_day2.take(3)

[('14.17.19.77', 1271), ('111.10.27.180', 2006), ('111.30.132.109', 16)]

In [78]:
server_downpacket_day1.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_downpacket/1228")
server_downpacket_day2.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_downpacket/1229")
server_downpacket_day3.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_downpacket/1230")

In [327]:
server_downpacket_day4.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_downpacket/1231")

In [135]:
server_downpacket_day1_dict = {}
for key, value in server_downpacket_day1.collect():
    server_downpacket_day1_dict[key] = value
server_downpacket_day2_dict = {}
for key, value in server_downpacket_day2.collect():
    server_downpacket_day2_dict[key] = value
server_downpacket_day3_dict = {}
for key, value in server_downpacket_day3.collect():
    server_downpacket_day3_dict[key] = value

In [328]:
server_downpacket_day4_dict = {}
for key, value in server_downpacket_day4.collect():
    server_downpacket_day4_dict[key] = value

In [168]:
tppd1_server_downpacket = train_positive_pairs_day1.map(lambda x: (x, server_downpacket_day1_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd2_server_downpacket = train_positive_pairs_day2.map(lambda x: (x, server_downpacket_day2_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)
tppd3_server_downpacket = train_positive_pairs_day3.map(lambda x: (x, server_downpacket_day3_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

In [169]:
tppd1_server_downpacket.take(3)

[('day1-8615150560352-111.30.128.27', [3886]),
 ('day1-8613813010581-117.135.169.19', [141772]),
 ('day1-8615951600106-117.135.169.19', [141772])]

In [329]:
test_ppd4_server_downpacket = test_positive_pairs_day4.map(lambda x: (x, server_downpacket_day4_dict[x.split('-')[2]]))\
.groupByKey().mapValues(list)

## merge feature

In [228]:
tppd1_features = sc.union([tppd1_user_brand, tppd1_user_activetime, tppd1_server_service,\
                           tppd1_server_subservice, tppd1_server_activetime,\
                           tppd1_user_diversity,tppd1_user_degree,tppd1_user_duration,\
                           tppd1_user_uppacket,tppd1_user_downpacket,\
                          tppd1_server_diversity,tppd1_server_degree,tppd1_server_duration,\
                           tppd1_server_uppacket,tppd1_server_downpacket])\
.groupByKey().mapValues(list)

tppd2_features = sc.union([tppd2_user_brand, tppd2_user_activetime, tppd2_server_service,\
                           tppd2_server_subservice, tppd2_server_activetime,\
                           tppd2_user_diversity,tppd2_user_degree,tppd2_user_duration,\
                           tppd2_user_uppacket,tppd2_user_downpacket,\
                          tppd2_server_diversity,tppd2_server_degree,tppd2_server_duration,\
                           tppd2_server_uppacket,tppd2_server_downpacket])\
.groupByKey().mapValues(list)

tppd3_features = sc.union([tppd3_user_brand, tppd3_user_activetime, tppd3_server_service,\
                           tppd3_server_subservice, tppd3_server_activetime,\
                           tppd3_user_diversity,tppd3_user_degree,tppd3_user_duration,\
                           tppd3_user_uppacket,tppd3_user_downpacket,\
                          tppd3_server_diversity,tppd3_server_degree,tppd3_server_duration,\
                           tppd3_server_uppacket,tppd3_server_downpacket])\
.groupByKey().mapValues(list)

In [222]:
print(tppd1_features.take(3))

[('day1-8615005274456-112.90.143.146', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [128, 0, 0, 0, 0, 0, 0, 24, 2, 103, 52, 20, 40, 13, 4, 20, 47, 61, 46, 78, 50, 66, 50, 566], [0, 0, 0, 0, 0, 0, 3, 0, 305, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 305, 0, 0, 0, 0, 0, 0, 0, 0, 0], [22, 2, 7, 2, 1, 0, 1, 4, 5, 8, 5, 16, 26, 13, 21, 18, 22, 22, 26, 10, 6, 36, 24, 11], [81], [312], [533688], [1652], [1533], [53], [94], [383197], [515], [389]]), ('day1-8615951015057-112.25.58.184', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6, 4, 4, 0, 0, 0, 0, 9, 35, 30, 0, 50, 3, 8, 277, 72, 14, 136, 54, 116, 6, 2, 10, 9], [0, 0, 0, 0, 0, 0, 1732

In [229]:
tppd1_features.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/tpp_features/1228")
tppd2_features.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/tpp_features/1229")
tppd3_features.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/tpp_features/1230")

In [330]:
test_ppd4_features = sc.union([test_ppd4_user_brand, test_ppd4_user_activetime, test_ppd4_server_service,\
                           test_ppd4_server_subservice, test_ppd4_server_activetime,\
                           test_ppd4_user_diversity,test_ppd4_user_degree,test_ppd4_user_duration,\
                           test_ppd4_user_uppacket,test_ppd4_user_downpacket,\
                          test_ppd4_server_diversity,test_ppd4_server_degree,test_ppd4_server_duration,\
                           test_ppd4_server_uppacket,test_ppd4_server_downpacket])\
.groupByKey().mapValues(list)

In [331]:
print(test_ppd4_features.take(3))

[('day4-8613913395739-223.85.133.151', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 1, 0, 5, 4, 8, 2, 7, 13, 2, 0, 5, 12, 2, 3, 17, 11, 3, 11], [0, 0, 0, 0, 0, 0, 11329, 0, 60, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11329, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0], [297, 139, 126, 109, 79, 97, 204, 430, 565, 591, 628, 690, 616, 615, 606, 599, 644, 693, 716, 729, 595, 626, 558, 437], [7], [40], [181016], [210], [215], [1847], [3947], [10613233], [17696], [14643]]), ('day4-8613951684695-113.108.67.102', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 0, 0, 0, 0, 6, 9, 8, 8, 10, 31, 17, 17, 16, 20, 60, 21, 24, 6, 26, 31, 19,

In [332]:
test_ppd4_features.coalesce(4)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/test_pp_features/1231")

In [371]:
train_up_others = train_unlabled_pairs.map(lambda x: (x,[[0] for i in range(10)]))

In [372]:
print(train_up_others.take(3))

[('day123-8615195968155-120.192.85.163', [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]), ('day123-8613770563324-180.153.82.27', [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]), ('day123-8613601430826-223.82.245.29', [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])]


In [401]:
train_unlabled_features = sc.union([train_up_user_brand, train_up_user_activetime, train_up_server_service,\
                           train_up_server_subservice, train_up_server_activetime,\
                           train_up_others])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [j for j in x[1][:-1]]+[i for i in x[1][-1]]))

In [402]:
print(train_unlabled_features.take(3))

[('day123-8615261441153-119.188.94.24', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 63, 8, 3, 18, 0, 8, 2, 15, 1, 8, 6, 1, 4], [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]), ('day123-8615861619510-111.4.118.162', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 14, 28, 36, 47, 102, 17, 60, 9, 16, 8, 0, 16, 0, 6, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [403]:
train_unlabled_features.coalesce(100)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_unlabled_features")

In [404]:
test_up_others = test_unlabled_pairs.map(lambda x: (x,[[0] for i in range(10)]))

In [405]:
test_unlabled_features = sc.union([test_up_user_brand, test_up_user_activetime, test_up_server_service,\
                           test_up_server_subservice, test_up_server_activetime,\
                           test_up_others])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [j for j in x[1][:-1]] + [i for i in x[1][-1]]))

In [406]:
print(test_unlabled_features.take(3))

[('day4-8613851678445-27.115.124.156', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 26, 26, 84, 15, 0, 14, 48, 15, 20, 9, 0, 63, 145, 28], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]), ('day4-8615240246213-111.1.21.49', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 16, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 

In [407]:
test_unlabled_features.coalesce(100)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/test_unlabled_features")

In [392]:
r1 = sc.parallelize([('a',[[0] for i in range(10)]),('b',[[0] for i in range(10)])])
r2 = sc.parallelize([('b',[1,2,3]),('a',[4,5,6])])
r3 = sc.parallelize([('b',5),('a',3)])

In [394]:
sc.union([r2,r3,r1]).groupByKey().mapValues(list).collect()

[('a', [[4, 5, 6], 3, [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]]),
 ('b', [[1, 2, 3], 5, [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]])]

In [ ]:
[[[0], [0]], [1, 2, 3]]

In [230]:
a = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [360, 294, 272, 290, 255, 260, 409, 291, 213, 86, 86, 59, 23, 308, 123, 167, 192, 170, 202, 338, 47, 15, 219, 134], [0, 0, 0, 0, 0, 0, 0, 0, 831, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 825, 0, 0], [47, 29, 12, 5, 13, 4, 26, 27, 39, 66, 65, 46, 60, 30, 30, 40, 39, 58, 52, 21, 46, 38, 20, 18], [36], [2153], [9018882], [13089], [9747], [94], [281], [1333259], [1953], [1869]]

In [231]:
aa = [item for i in a for item in i]

In [234]:
print(aa)
print(len(aa))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 360, 294, 272, 290, 255, 260, 409, 291, 213, 86, 86, 59, 23, 308, 123, 167, 192, 170, 202, 338, 47, 15, 219, 134, 0, 0, 0, 0, 0, 0, 0, 0, 831, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 825, 0, 0, 47, 29, 12, 5, 13, 4, 26, 27, 39, 66, 65, 46, 60, 30, 30, 40, 39, 58, 52, 21, 46, 38, 20, 18, 36, 2153, 9018882, 13089, 9747, 94, 281, 1333259, 1953, 1869]
176


In [103]:
sc.union([tppd1_user_duration,tppd1_user_degree,tppd1_user_diversity]).groupByKey().mapValues(list).take(3)

[('day1-8615150560352-111.30.128.27', [[560262], [74], [29]]),
 ('day1-8613813010581-117.135.169.19', [[168686], [72], [12]]),
 ('day1-8615951600106-117.135.169.19', [[133647], [124], [3]])]

[[0], [0], [0]]